In [ ]:
!pip3

In [15]:
import pandas as pd
import numpy as np
import os


filename = "heavy_data.csv"
if not os.path.exists(filename):
    print("Генерируем данные...")
    df_dummy = pd.DataFrame(np.random.randint(0, 100, size=(3000000, 5)), columns=list('ABCDE'))
    df_dummy.to_csv(filename, index=False)
    print(f"Готово! Файл {filename} создан.")
else:
    print("Файл уже есть.")

%load_ext memory_profiler

Файл уже есть.
The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [16]:
%%writefile pandas_memory_test.py
from memory_profiler import profile
import pandas as pd
import time

# Этот декоратор теперь сработает, потому что мы запустим это как внешний скрипт
@profile
def load_heavy_process():
    print("--- Шаг 1: Старт функции ---")
    time.sleep(0.5)
    
    print("--- Шаг 2: Загрузка CSV (Смотри на скачок памяти ниже!) ---")
    # Читаем целиком
    df = pd.read_csv("heavy_data.csv")
    
    print("--- Шаг 3: Агрегация ---")
    # Какая-нибудь операция
    res = df.groupby('A').sum()
    
    print("--- Шаг 4: Конец ---")
    return res

if __name__ == "__main__":
    load_heavy_process()


Overwriting pandas_memory_test.py


In [17]:
!python3 -m memory_profiler pandas_memory_test.py


--- Шаг 1: Старт функции ---
--- Шаг 2: Загрузка CSV (Смотри на скачок памяти ниже!) ---
--- Шаг 3: Агрегация ---
--- Шаг 4: Конец ---
Filename: pandas_memory_test.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     6     74.5 MiB     74.5 MiB           1   @profile
     7                                         def load_heavy_process():
     8     74.5 MiB      0.0 MiB           1       print("--- Шаг 1: Старт функции ---")
     9     74.5 MiB      0.0 MiB           1       time.sleep(0.5)
    10                                             
    11     74.5 MiB      0.0 MiB           1       print("--- Шаг 2: Загрузка CSV (Смотри на скачок памяти ниже!) ---")
    12                                             # Читаем целиком
    13    441.4 MiB    366.9 MiB           1       df = pd.read_csv("heavy_data.csv")
    14                                             
    15    441.4 MiB      0.0 MiB           1       print("--- Шаг 3: Агрегация ---")
    16                

In [18]:
%%writefile pandas_chunk_test.py
from memory_profiler import profile
import pandas as pd

@profile
def load_with_chunks():
    print("--- Читаем по кусочкам ---")
    chunk_size = 500000 
    total_sum = 0
    
    # Итератор вместо загрузки всего файла
    for chunk in pd.read_csv("heavy_data.csv", chunksize=chunk_size):
        # Память растет только на размер чанка, потом очищается
        total_sum += chunk['A'].sum()
        
    print(f"Сумма: {total_sum}")

if __name__ == "__main__":
    load_with_chunks()


Overwriting pandas_chunk_test.py


In [19]:
!python3 -m memory_profiler pandas_chunk_test.py


--- Читаем по кусочкам ---
Сумма: 148461287
Filename: pandas_chunk_test.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     4     74.7 MiB     74.7 MiB           1   @profile
     5                                         def load_with_chunks():
     6     74.7 MiB      0.0 MiB           1       print("--- Читаем по кусочкам ---")
     7     74.7 MiB      0.0 MiB           1       chunk_size = 500000 
     8     74.7 MiB      0.0 MiB           1       total_sum = 0
     9                                             
    10                                             # Итератор вместо загрузки всего файла
    11    261.4 MiB    186.5 MiB           7       for chunk in pd.read_csv("heavy_data.csv", chunksize=chunk_size):
    12                                                 # Память растет только на размер чанка, потом очищается
    13    261.4 MiB      0.2 MiB           6           total_sum += chunk['A'].sum()
    14                                                 


In [ ]:
import csv

def hadoop_simulation():
    # ЭТАП 1: MAPPER (Читает диск -> Пишет на диск)
    # Hadoop не может передать данные в памяти между этапами, он сохраняет их
    print("Starting Mapper Job...")
    with open("big_data.csv", "r") as infile, open("intermediate_data.csv", "w") as outfile:
        reader = csv.DictReader(infile)
        writer = csv.writer(outfile)
        
        for row in reader:
            if row['category'] == 'A': # Фильтрация
                # Пишем на диск! (Самая медленная операция в мире компьютеров)
                writer.writerow([row['category'], row['A']])

    # ЭТАП 2: SHUFFLE & SORT (Сортировка на диске)
    # (Опустим код, но представьте, что здесь происходит сортировка файлов)

    # ЭТАП 3: REDUCER (Читает диск -> Пишет на диск)
    print("Starting Reducer Job...")
    total_sum = 0
    with open("intermediate_data.csv", "r") as infile:
        reader = csv.reader(infile)
        for row in reader:
            # Снова читаем с диска!
            total_sum += int(row[1])
            
    print(f"Result: {total_sum}")
    
    # Очистка мусора (Hadoop это делает сам, но ценой времени)
    os.remove("intermediate_data.csv")

if __name__ == "__main__":
    hadoop_simulation()


In [21]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

def process_with_spark():
    spark = SparkSession.builder \
        .appName("SparkHero") \
        .master("local[*]") \
        .config("spark.driver.memory", "2g") \
        .getOrCreate()

    # 1. Lazy Loading (Мгновенно, память не ест)
    # Spark не читает файл сейчас. Он создает план.
    df = spark.read.csv("big_data.csv", header=True, inferSchema=True)

    # 2. Transformations (Цепочка в памяти)
    # В отличие от Hadoop, здесь не создаются промежуточные файлы на диске.
    # Spark объединит filter и groupBy в один проход (Pipelining).
    result_df = df.filter(F.col("category") == "A") \
                  .groupBy("category") \
                  .agg(F.sum("A").alias("total"))

    # 3. Action
    # Только здесь начнется работа.
    # Spark разобьет файл на партиции (как мы делали чанки в Pandas),
    # Раздаст их ядрам (local[*]),
    # Посчитает всё в RAM и вернет результат.
    result_df.show()
    
    # Демонстрация плана выполнения (Доказательство ума Spark)
    result_df.explain()

if __name__ == "__main__":
    process_with_spark()


The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.

/Users/ivan7chuk/Library/Python/3.9/lib/python/site-packages/pyspark/bin/spark-class: line 97: CMD: bad array subscript
head: illegal line count -- -1


PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [ ]:
# Сложная цепочка
df1 = df.withColumn("x", F.col("A") * 2)
df2 = df1.filter(F.col("x") > 10)
df3 = df2.groupBy("category").count()

# В Hadoop это было бы 3 Job-ы (чтение-запись-чтение-запись...).
# В Spark это ОДИН проход по данным.
# Catalyst Optimizer видит: "Ага, умножение и фильтр можно сделать одновременно, пока строка лежит в кэше процессора".


In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, StringType, StructType, StructField, IntegerType

# 1. Генерируем сложные данные (Заказ -> Список товаров)
data_nested = [
    (1, "Vasya", ["apple", "banana", "coffee"]),
    (2, "Petya", ["pizza"]),
    (3, "Oleg", []) # Пустой список
]

df_nested = spark.createDataFrame(data_nested, ["order_id", "user", "items"])

print("Исходные данные (массив внутри ячейки):")
df_nested.show(truncate=False)

# 2. EXPLODE (Магия Spark)
# Превращаем массив в строки. Одна строка заказа превратится в три строки товаров.
# Это стандартная операция нормализации логов.
df_exploded = df_nested.select(
    "order_id", 
    "user", 
    F.explode("items").alias("item") # explode уничтожает строки с пустыми массивами!
)

# for row in rows:
#     for elem in row['items']:
#         new_row = row
#         row['item'] = elem
#         yield new_row
# Хинт: если нужны и пустые массивы, используйте explode_outer

print("После explode (Нормализованная таблица):")
df_exploded.show()

# 3. Агрегация обратно (Collect List)
# Допустим, мы пофильтровали товары и хотим свернуть всё обратно в список
df_collapsed = df_exploded.groupBy("order_id", "user") \
    .agg(F.collect_list("item").alias("items_list"))
# for group in groups:
#     order_id = None
#     user = None
#     items = []
#     for row in rows:
#         order_id = row['order_id']
#         user = row['user']
#         items.append(row['item'])
#     new_row = {
#         'order_id': order_id,
#         'user': user,
#         'items_list': items
#     }
#     yield new_row
print("Собрали обратно:")
df_collapsed.show(truncate=False)


In [ ]:
from pyspark.sql.window import Window

# Данные: Продажи по дням и отделам
data_sales = [
    ("Dep_A", "2023-01-01", 100),
    ("Dep_A", "2023-01-02", 150), # Рост
    ("Dep_A", "2023-01-03", 120), # Падение
    ("Dep_B", "2023-01-01", 500),
    ("Dep_B", "2023-01-02", 500), # Стабильность
    ("Dep_B", "2023-01-03", 600)
]
df_sales = spark.createDataFrame(data_sales, ["dept", "date", "sales"])

# ОПРЕДЕЛЯЕМ ОКНО
# "Для каждого отдела, отсортировав по дате..."
window_spec = Window.partitionBy("dept").orderBy("date")

df_analytics = df_sales.withColumn(
    "prev_day_sales", 
    F.lag("sales", 1).over(window_spec) # LAG - взять значение из предыдущей строки
).withColumn(
    "diff", 
    F.col("sales") - F.col("prev_day_sales") # Разница
).withColumn(
    "running_total",
    F.sum("sales").over(window_spec) # Нарастающий итог (Running Total)
)

print("Аналитика изменений (Lag и Running Total):")
df_analytics.show()


In [20]:
# Грязные данные: Цена как строка, даты кривые
data_dirty = [
    ("TV", "50000.00", "2023-01-01"),
    ("Phone", "null", "01/01/2023"), # Другой формат даты
    ("Laptop", "100k", "2023-Hello"), # Мусор
    ("Mouse", "1500", "2023-01-10"),
]
df_dirty = spark.createDataFrame(data_dirty, ["product", "price_str", "date_str"])

# Очистка
df_clean = df_dirty.withColumn(
    "price", 
    F.col("price_str").cast("double") # Безопасный каст (мусор станет null)
).withColumn(
    "date_unified",
    # COALESCE - верни первое не null значение
    F.coalesce(
        F.to_date("date_str", "yyyy-MM-dd"), # Пробуем формат 1
        F.to_date("date_str", "dd/MM/yyyy")  # Пробуем формат 2
    )
).withColumn(
    "is_valid",
    # SQL выражение прямо в коде! Очень удобно для сложной логики.
    F.expr("price IS NOT NULL AND date_unified IS NOT NULL")
)

print("Очистка и валидация:")
df_clean.show()


NameError: name 'spark' is not defined

In [ ]:
# ПЛОХОЙ ПРИМЕР: Итерация через Python (Looping)
def anti_pattern_demo():
    # .collect() забирает ВСЕ данные на драйвер (ваш ноутбук)
    rows = df_sales.collect() 
    
    # Loop происходит на ОДНОМ ядре (на драйвере)
    # Вся мощь кластера простаивает!
    results = []
    for row in rows:
        # Медленная питоновская логика
        if row['sales'] > 100:
            results.append(row['dept'] + "_high")
        else:
            results.append(row['dept'] + "_low")
            
    print(results[:5])

# ХОРОШИЙ ПРИМЕР: Векторная операция
def good_pattern_demo():
    # Работает на кластере, параллельно, без загрузки на драйвер
    df_sales.withColumn(
        "tag", 
        F.when(F.col("sales") > 100, F.concat(F.col("dept"), F.lit("_high")))
         .otherwise(F.concat(F.col("dept"), F.lit("_low")))
    ).show()

good_pattern_demo()


In [ ]:
# Данные: Продажи по магазинам и месяцам
data_pivot = [
    ("Shop_1", "Jan", 100),
    ("Shop_1", "Feb", 150),
    ("Shop_1", "Mar", 120),
    ("Shop_2", "Jan", 200),
    ("Shop_2", "Feb", 210),
]
df_pivot = spark.createDataFrame(data_pivot, ["shop", "month", "amount"])

print("Исходная (Long format):")
df_pivot.show()

# PIVOT (Строки -> Колонки)
# Для каждого магазина сделать колонки Jan, Feb, Mar с суммой продаж
df_pivoted = df_pivot.groupBy("shop") \
    .pivot("month", ["Jan", "Feb", "Mar"]) \
    .sum("amount") \
    .na.fill(0) # Заменяем null на 0, если данных не было

print("Повернутая (Wide format):")
df_pivoted.show()


In [ ]:
df_dates = spark.createDataFrame([("2023-01-31",), ("2023-02-28",)], ["date_str"])

df_dates_calc = df_dates.select(
    F.col("date_str"),
    # 1. Сложение дат
    F.date_add(F.col("date_str"), 10).alias("plus_10_days"),
    F.add_months(F.col("date_str"), 1).alias("plus_1_month"), # Умное сложение (учитывает длину месяца)
    
    # 2. Текущая дата и разница
    F.current_date().alias("today"),
    F.datediff(F.current_date(), F.col("date_str")).alias("days_passed"),
    
    # 3. Trunc (Округление до начала месяца/года)
    F.trunc(F.col("date_str"), "month").alias("start_of_month"),
    
    # 4. Форматирование
    F.date_format(F.col("date_str"), "dd.MM.yyyy -- EEEE").alias("pretty_date")
)

df_dates_calc.show(truncate=False)


In [ ]:
# Создадим "плохой" CSV файл программно
bad_csv_content = """id,name,age
1,Vasya,25
2,Petya,Corrupted_Age
3,Oleg,30
4,Broken_Row
"""
with open("bad_data.csv", "w") as f:
    f.write(bad_csv_content)

print("Читаем плохой файл...")

# Режим PERMISSIVE (по умолчанию) с колонкой _corrupt_record
df_bad = spark.read \
    .option("header", "true") \
    .option("mode", "PERMISSIVE") \
    .option("columnNameOfCorruptRecord", "_raw_error") \
    .csv("bad_data.csv")

# Что произошло:
# 1. Хорошие строки распарсились.
# 2. Плохие строки попали в _raw_error.
# 3. Остальные поля в плохих строках стали null.
df_bad.show()

# Как это чистить в проде:
clean_df = df_bad.filter(F.col("_raw_error").isNull()).drop("_raw_error")
error_df = df_bad.filter(F.col("_raw_error").isNotNull())

print("Чистые данные:")
clean_df.show()


In [ ]:
# Генерируем данные с повторами (1 млн строк)
big_range = spark.range(0, 1000000).withColumn("group_id", F.lit(1))
# Дублируем, чтобы проверить distinct
big_data = big_range.union(big_range) 

start = time.time()
exact = big_data.select(F.countDistinct("id")).collect()[0][0]
print(f"Точный подсчет: {exact}, Время: {time.time() - start:.2f} сек")

start = time.time()
# rsd - максимальная относительная ошибка. 0.05 = 5%
approx = big_data.select(F.approx_count_distinct("id", rsd=0.05)).collect()[0][0]
print(f"Быстрый подсчет: {approx}, Время: {time.time() - start:.2f} сек")


In [ ]:
data_users = [(1, "user", 50), (2, "admin", 0), (3, "user", 1000), (4, "guest", 0)]
df_users = spark.createDataFrame(data_users, ["id", "role", "spent"])

# Сложная логика тегирования
df_tagged = df_users.withColumn("segment", 
    F.when(F.col("role") == "admin", "Internal")
     .when((F.col("role") == "user") & (F.col("spent") > 500), "Premium User")
     .when(F.col("spent") > 0, "Active User")
     .otherwise("Ghost")
)

df_tagged.show()
